In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
# tf.config.run_functions_eagerly(True)

from pyspark.sql import SparkSession
import numpy as np
import autokeras as ak

spark = SparkSession \
    .builder \
    .appName("Cerebro Iris") \
    .getOrCreate()

...

backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

from pyspark.ml.feature import OneHotEncoderEstimator

df = spark.read.csv("/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/Iris_clean.csv", header=True, inferSchema=True)

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["Species"])
encoder.setOutputCols(["Species_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
label_columns=['Species_OHE']

# Initialize input DataFrames.
# You can download sample dataset from https://apache.googlesource.com/spark/+/master/data/mllib/sample_libsvm_data.txt

train_df, test_df = encoded.randomSplit([0.8, 0.2], 100)

21/11/29 02:01:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/29 02:01:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


CEREBRO => Time: 2021-11-29 02:01:23, Running 1 Workers


In [2]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.model_selection import train_test_split

# pddf = df.toPandas()
# train_df, test_df = train_test_split(pddf, test_size=0.2, random_state=42, shuffle=True)
# train_df, test_df = df.randomSplit([0.8, 0.2])
# train = train_df.toPandas()
# test = test_df.toPandas()

# x_train = train_df[feature_columns]
# y_train = train_df[['Species']]

# enc = OneHotEncoder()
# x = np.array(x_train)
# x = [x[:,i,np.newaxis] for i in range(x.shape[1])]
# y = enc.fit_transform(y_train).toarray()

# x_test = test_df[feature_columns]
# y_test = test_df[['Species']]

# y_test = enc.fit_transform(y_test).toarray()
# x_test = np.array(x_test)
# x_test = [x_test[:,i,np.newaxis] for i in range(x_test.shape[1])]

In [6]:
img_shape = (28,28,1)

x = np.array(train_df.select(feature_columns).head(100))
y = np.array(train_df.select(label_columns).head(100))
x = [x[:,i] for i in range(x.shape[1])]
x = [r.reshape((-1, *img_shape)) for r in x]
y = np.squeeze(y,1)
print(x.shape)

x_test = np.array(test_df.select(feature_columns).collect())
y_test = np.array(test_df.select(label_columns).collect())
x_test = [x_test[:,i,np.newaxis] for i in range(x_test.shape[1])]
y_test = np.squeeze(y_test,1)
print(y_test.shape)

ValueError: cannot reshape array of size 100 into shape (28,28,1)

In [2]:
x = np.array(train_df.select(feature_columns).head(100))
x = [x[:,i] for i in range(x.shape[1])]
x[0].shape

(100,)

In [4]:
input_node = [ak.StructuredDataInput() for c in feature_columns]
# embeddings = [ak.StructuredDataBlock()(innode) for innode in input_node]
# output_node = ak.Merge()([embeddings])
concat = ak.Merge()(input_node)
output_node = ak.DenseBlock()(concat)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node, 
    max_trials=20, 
    tuner='random',
    objective="val_loss",
    seed=100,
    overwrite=True
)

In [5]:
# dataset, validation_data = clf._convert_to_dataset(
#             x=x, y=y, validation_data=None, batch_size=32
#         )
# clf._analyze_data(dataset)
# clf._build_hyper_pipeline(dataset)

In [6]:
# hp = clf.tuner.oracle.get_space()
# clf.tuner._prepare_model_build(hp, x=dataset)
# clf.tuner.hypermodel.build(hp)
# clf.tuner.oracle.update_space(hp)

In [7]:
# trial = clf.tuner.oracle.create_trial(clf.tuner.tuner_id)
# trial.hyperparameters.values

In [8]:
# clf.tuner.oracle._tried_so_far

In [9]:
# clf.tuner.oracle.get_space().space

In [10]:
clf.fit(
    x,
    y,
    # The name of the label column.
    epochs=10,
)

Trial 20 Complete [00h 00m 02s]
val_loss: 0.5947383046150208

Best val_loss So Far: 0.36133790016174316
Total elapsed time: 00h 00m 59s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
4/4 [==============================] - 0s 5ms/step - loss: 1.9923 - accuracy: 0.3250
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 1.2646 - accuracy: 0.3123
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 0.9194 - accuracy: 0.5104
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6687 - accuracy: 0.7100
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6019 - accuracy: 0.8075
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6156 - accuracy: 0.7454
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4541 - accuracy: 0.8617
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7746
Epoch 9/10
4/4 [==============================] - 0s 4ms/s

2021-11-22 06:02:18.877548: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [11]:
model = clf.tuner.get_best_model()

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [13]:
clf.evaluate(x_test, y_test)

1/1 [==============================] - 0s 186ms/step - loss: 0.2815 - accuracy: 0.8667


[0.28146201372146606, 0.8666666746139526]

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers

import autokeras as ak

In [13]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [14]:
# from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder()
# y_train = enc.fit_transform(y_train).toarray()
# y_test = enc.fit_transform(y_test).toarray()

# x_val = x_train[-200:]
# y_val = y_train[-200:]
# x_train = x_train[:600]
# y_train = y_train[:600]
print(y_train.shape)

(60000, 10)


In [15]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [16]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

2021-11-28 17:43:56.804201: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/15
422/422 [==============================] - 12s 27ms/step - loss: 0.7651 - accuracy: 0.7605 - val_loss: 0.0837 - val_accuracy: 0.9777
Epoch 2/15
422/422 [==============================] - 14s 32ms/step - loss: 0.1232 - accuracy: 0.9622 - val_loss: 0.0568 - val_accuracy: 0.9852
Epoch 3/15
422/422 [==============================] - 15s 36ms/step - loss: 0.0918 - accuracy: 0.9718 - val_loss: 0.0484 - val_accuracy: 0.9848
Epoch 4/15
422/422 [==============================] - 15s 36ms/step - loss: 0.0742 - accuracy: 0.9760 - val_loss: 0.0437 - val_accuracy: 0.9887
Epoch 5/15
422/422 [==============================] - 16s 37ms/step - loss: 0.0657 - accuracy: 0.9794 - val_loss: 0.0399 - val_accuracy: 0.9888
Epoch 6/15
422/422 [==============================] - 15s 34ms/step - loss: 0.0553 - accuracy: 0.9826 - val_loss: 0.0373 - val_accuracy: 0.9898
Epoch 7/15
422/422 [==============================] - 15s 35ms/step - loss: 0.0518 - accuracy: 0.9830 - val_loss: 0.0362 - val_accuracy:

In [17]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.0243 - accuracy: 0.9915


[0.02429737150669098, 0.9915000200271606]

In [21]:
# input_node = ak.ImageInput()
# output_node = ak.ImageBlock(
#     # Only search ResNet architectures.
#     block_type="resnet",
#     # Normalize the dataset.
#     normalize=True,
#     # Do not do data augmentation.
#     augment=False,
# )(input_node)
# output_node = ak.ClassificationHead()(output_node)
# clf = ak.AutoModel(
#     inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
# )
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
clf.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Trial 1 Complete [00h 04m 44s]
val_loss: 0.04536424204707146

Best val_loss So Far: 0.04536424204707146
Total elapsed time: 00h 04m 44s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
938/938 [==============================] - 65s 69ms/step - loss: 0.3546 - accuracy: 0.8903
Epoch 2/5
938/938 [==============================] - 66s 70ms/step - loss: 0.0796 - accuracy: 0.9751
Epoch 3/5
938/938 [==============================] - 66s 71ms/step - loss: 0.0619 - accuracy: 0.9806
Epoch 4/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0502 - accuracy: 0.9838
Epoch 5/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0452 - accuracy: 0.9855


2021-11-28 19:30:56.681247: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [19]:
clf.evaluate(x_test, y_test)

NotFoundError: ./auto_model/best_pipeline; No such file or directory

In [9]:
model = clf.tuner.get_best_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 28, 28, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           cast_to_float32[0][0]            
__________________________________________________________________________________________________
resizing (Resizing)             (None, 32, 32, 1)    0           normalization[0][0]              
______________________________________________________________________________________________